# Simon's world Cup 2018 Playground

This is the playground of mine to play with world cup 2018's data. The data is based on [this free API](http://worldcup.sfg.io/). This notebook is the testfield and lab where I can play, explore and learn from this project of mine, so please bear with me if it's too messy.

In [3]:
import json
import urllib.request

group_dict = {}
team_dict = {}
match_dict = {}

### classes of the project

In [4]:
class wc_group(object):
    def __init__(self, group_id, letter, teams):
        self.group_id = group_id
        self.letter = letter
        self.teams = teams
        
    def __repr__(self):
        return "Group: " + self.letter
    
    def teamInGroup(self):
        for team in self.teams.values():
            yield team
        
class wc_team(object):

    def __init__(self, team_id, country, fifa_code, wins, draws, losses, games_played, points, goals_for, goals_against, goal_differential):
        self.team_match_dict = {}
        self.team_completed_match_dict = {}
        self.team_win_match_dict = {}
        self.team_draw_match_dict = {}
        self.team_loss_match_dict = {}
        self.match_id_opponent_dict = {}
        self.team_id = team_id
        self.country = country
        self.fifa_code = fifa_code
        self.wins = wins
        self.draws = draws
        self.losses = losses
        self.games_played = games_played
        self.points = points
        self.goals_for = goals_for
        self.goals_against = goals_against
        self.goal_differential = goal_differential
        
        
    def __repr__(self):
        return self.country + "(" + self.fifa_code + ") Wins:" + str(self.wins) + " Draws:" + str(self.draws) +\
               " Losses:" + str(self.losses) + " Games played:" + str(self.games_played) + " Points:" + str(self.points) +\
               " Goals for:" + str(self.goals_for) + " Goals against:" + str(self.goals_against) + " Goal differential:" +\
               str(self.goal_differential)
                
    def _classifyPlayedMatchesByResults(self):
        for m in self.team_completed_match_dict.values():
            if m.getWinner() == "Draw":
                self.team_draw_match_dict[m.getMatchId()] = m
            elif m.getWinner() == self:
                self.team_win_match_dict[m.getMatchId()] = m
            else:
                self.team_loss_match_dict[m.getMatchId()] = m
                        
    def fetchDetailedMatchInfo(self):
        # this function will add all detailed info of all matches of this team
        matches_json = urllib.request.urlopen('http://worldcup.sfg.io/matches/country?fifa_code=' + self.fifa_code)
        matches_python = json.loads(matches_json.read())
        
        for m in matches_python:
            if not m['fifa_id'] in match_dict:
                match_dict[m['fifa_id']] = wc_match(location = m['venue'], venue = m['location'],
                                                    status = m['status'], time = m['time'],
                                                    fifa_id = m['fifa_id'], datetime = m['datetime'],
                                                    home_team = team_dict[m['home_team']['country']],
                                                    away_team = team_dict[m['away_team']['country']],
                                                    home_team_goals = m['home_team']['goals'],
                                                    away_team_goals = m['away_team']['goals'],
                                                    home_team_statistics = m['home_team_statistics'],
                                                    away_team_statistics = m['away_team_statistics'],
                                                    home_event_list = m['home_team_events'],
                                                    away_event_list = m['away_team_events'], winner = m['winner_code'])
            if not m['fifa_id'] in self.match_id_opponent_dict:
                if m['home_team']['country'] == self.country:
                    self.match_id_opponent_dict[m['fifa_id']] = team_dict[m['away_team']['country']]
                else:
                    self.match_id_opponent_dict[m['fifa_id']] = team_dict[m['home_team']['country']]
                    
            if not m['fifa_id'] in self.team_match_dict:
                self.team_match_dict[m['fifa_id']] = match_dict[m['fifa_id']]
            if match_dict[m['fifa_id']].isCompleted():
                if not m['fifa_id'] in self.team_completed_match_dict:
                    self.team_completed_match_dict[m['fifa_id']] = match_dict[m['fifa_id']]
            
        
        matches_json.close()
        
        self._classifyPlayedMatchesByResults()
    
    def getFifaCode(self):
        return self.fifa_code
    
    def getCountryName(self):
        return self.country
    
    def getCountryNameWithCode(self):
        return self.country + "(" + self.fifa_code + ")"
    
    def generateCompletedMatches(self):
        for match in self.team_completed_match_dict.values():
            yield match
            
    def getMatchDictByOpponent(self):
        d = {}
        for m in self.team_match_dict.values():
            if m.getHomeTeam() == self:
                d["v.s. " + m.getAwayTeam().getCountryName() + "   [" + m.getMatchId() + "]"] = m
            else:
                d["v.s. " + m.getHomeTeam().getCountryName() + "   [" + m.getMatchId() + "]"] = m
        return d
                
    
class wc_match(object):
    def __init__(self, location, venue, status , time, fifa_id, datetime ,
                 home_team_goals, away_team_goals,
                 home_team_statistics, away_team_statistics,
                 home_team, away_team, home_event_list, away_event_list, winner):
        self.location = location
        self.venue = venue
        self.status = status
        self.time = time
        self.fifa_id = fifa_id
        self.datetime = datetime
        self.home_team_goals = home_team_goals
        self.away_team_goals = away_team_goals
        self.home_team_statistics = home_team_statistics
        self.away_team_statistics = away_team_statistics
        self.home_team = home_team
        self.away_team = away_team
        self.home_event_list = home_event_list
        for d in self.home_event_list:
            d['team'] = home_team
        self.away_event_list = away_event_list
        for d in self.away_event_list:
            d['team'] = away_team
        if self.status=='completed':
            if winner != 'Draw':
                self.winner = self.home_team if self.home_team.getFifaCode() == winner else self.away_team
            else:
                self.winner = "Draw"
            self._buildOrderedEventList()
        else:
            self.ordered_event_list = []
            self.winner = ""
        
    def __repr__(self):
        return "Match " + self.fifa_id + ": " + self.home_team.getCountryNameWithCode() + " v.s. " +\
               self.away_team.getCountryNameWithCode() + " on " + self.datetime + " in " + self.venue
        
    def _buildOrderedEventList(self):
        from operator import itemgetter
        e_list = self.home_event_list + self.away_event_list
        self.ordered_event_list = sorted(e_list, key=itemgetter("time","id"))
    
    def getMatchId(self):
        return self.fifa_id
    
    def isCompleted(self):
        return True if self.status == 'completed' else False
    
    def getWinner(self):
        return self.winner
    
    def getHomeTeam(self):
        return self.home_team
    
    def getAwayTeam(self):
        return self.away_team
    
    def printBasicMatchInfo(self):
        print("Venue:          " + self.venue + ", " + self.location)
        print("Date Time:      " + self.datetime)
        if len(self.home_team.getCountryNameWithCode())<15:
            blank = ' '*(16 - len(self.home_team.getCountryNameWithCode()))
        else:
            blank = ' '*2
        print(self.home_team.getCountryNameWithCode() + blank + "v.s." + blank + self.away_team.getCountryNameWithCode())
        if self.status == 'completed':
            print(' '*8 + str(self.home_team_goals) + 8*' ' + '--' + 8*' ' + str(self.away_team_goals)) 
            if self.winner != 'Draw':             
                print(self.winner.getCountryNameWithCode() + " wins!")
            else:
                print("Draw Game!")
        else:
            print("Game is not yet finished!")
    
    def printMatchEvents(self):
        print("Events:")
        for d in self.ordered_event_list:
            print(d)
            
    def getOrderedEventsList(self):
        return self.ordered_event_list

### build the dict of group and the dict of team

In [5]:
group_results_json = urllib.request.urlopen('http://worldcup.sfg.io/teams/group_results')
group_results_python = json.loads(group_results_json.read())

for group in group_results_python:
    team_dict_in_group = {}
    for team in group['group']['teams']:
        team_dict_in_group[team['team']['country']] = wc_team(team['team']['id'], team['team']['country'], team['team']['fifa_code'],
                                                              team['team']['wins'], team['team']['draws'], team['team']['losses'],
                                                              team['team']['games_played'], team['team']['points'], team['team']['goals_for'],
                                                              team['team']['goals_against'], team['team']['goal_differential'])
        team_dict[team['team']['country']] = team_dict_in_group[team['team']['country']]
    group_dict[group['group']['letter']] = wc_group(group['group']['id'], group['group']['letter'], team_dict_in_group)

In [6]:
# print them
for g in group_dict.values():
    print(g)
    for t in g.teamInGroup():
        print(t)

Group: A
Russia(RUS) Wins:2 Draws:0 Losses:0 Games played:2 Points:6 Goals for:8 Goals against:1 Goal differential:7
Uruguay(URU) Wins:2 Draws:0 Losses:0 Games played:2 Points:6 Goals for:2 Goals against:0 Goal differential:2
Egypt(EGY) Wins:0 Draws:0 Losses:2 Games played:2 Points:0 Goals for:1 Goals against:4 Goal differential:-3
Saudi Arabia(KSA) Wins:0 Draws:0 Losses:2 Games played:2 Points:0 Goals for:0 Goals against:6 Goal differential:-6
Group: B
Spain(ESP) Wins:1 Draws:1 Losses:0 Games played:2 Points:4 Goals for:4 Goals against:3 Goal differential:1
Portugal(POR) Wins:1 Draws:1 Losses:0 Games played:2 Points:4 Goals for:4 Goals against:3 Goal differential:1
Iran(IRN) Wins:1 Draws:0 Losses:1 Games played:2 Points:3 Goals for:1 Goals against:1 Goal differential:0
Morocco(MAR) Wins:0 Draws:0 Losses:2 Games played:2 Points:0 Goals for:0 Goals against:2 Goal differential:-2
Group: C
Denmark(DEN) Wins:1 Draws:1 Losses:0 Games played:2 Points:4 Goals for:2 Goals against:1 Goal differ

### build all matches in group A

In [7]:
for t in group_dict['A'].teamInGroup():
    t.fetchDetailedMatchInfo()

### build all matches in group B

In [8]:
for t in group_dict['B'].teamInGroup():
    t.fetchDetailedMatchInfo()

### print all completed matches of Russia in Group A

In [9]:
for m in team_dict['Russia'].generateCompletedMatches():
    print(m)
    m.printBasicMatchInfo()
    m.printMatchEvents()

Match 300331503: Russia(RUS) v.s. Saudi Arabia(KSA) on 2018-06-14T15:00:00Z in Luzhniki Stadium
Venue:          Luzhniki Stadium, Moscow
Date Time:      2018-06-14T15:00:00Z
Russia(RUS)     v.s.     Saudi Arabia(KSA)
        5        --        0
Russia(RUS) wins!
Events:
{'id': 5, 'type_of_event': 'goal', 'player': 'Iury GAZINSKY', 'time': "12'", 'team': Russia(RUS) Wins:2 Draws:0 Losses:0 Games played:2 Points:6 Goals for:8 Goals against:1 Goal differential:7}
{'id': 1, 'type_of_event': 'substitution-in', 'player': 'Denis CHERYSHEV', 'time': "24'", 'team': Russia(RUS) Wins:2 Draws:0 Losses:0 Games played:2 Points:6 Goals for:8 Goals against:1 Goal differential:7}
{'id': 320, 'type_of_event': 'substitution-out', 'player': 'Alan DZAGOEV', 'time': "24'", 'team': Russia(RUS) Wins:2 Draws:0 Losses:0 Games played:2 Points:6 Goals for:8 Goals against:1 Goal differential:7}
{'id': 2, 'type_of_event': 'goal', 'player': 'Denis CHERYSHEV', 'time': "43'", 'team': Russia(RUS) Wins:2 Draws:0 Losses

### interactive with *ipwidgets*

In [11]:
import ipywidgets as widgets
out = widgets.Output(layout={'border': '3px solid black'})
match_info_out = widgets.Output(layout={'border': '1px solid black'})


def showMatchInfo(x):
    match_info_out.clear_output()
    with match_info_out:
        x.printBasicMatchInfo()
    display(match_info_out)
    button_show_events = widgets.Button(description="Show Events")
    button_home_statistics = widgets.Button(description = x.getHomeTeam().getCountryName() + " statistics")
    button_away_statistics = widgets.Button(description = x.getAwayTeam().getCountryName() + " statistics")
    bout=widgets.Output()
    def buttonShowEventsOnClick(b):
        bout.clear_output()
        with bout:
            for e in x.getOrderedEventsList():
                print(e)
    def buttonShowHomeStatisticsOnClick(b):
        bout.clear_output()
        with bout:
            for k, v in x.home_team_statistics.items():
                print(k + " " + str(v))
    def buttonShowAwayStatisticsOnClick(b):
        bout.clear_output()
        with bout:
            for k, v in x.away_team_statistics.items():
                print(k + " " + str(v))
    button_show_events.on_click(buttonShowEventsOnClick)
    button_home_statistics.on_click(buttonShowHomeStatisticsOnClick)
    button_away_statistics.on_click(buttonShowAwayStatisticsOnClick)
    if x.isCompleted():
        #display(button_show_events)
        #display(button_home_statistics)
        #display(button_away_statistics)
        #display(bout)
        buttons_results_box = widgets.VBox([widgets.HBox([button_show_events, button_home_statistics, button_away_statistics]), bout])
        display(buttons_results_box)

def showTeamInfo(x):
    out.clear_output()
    with out:
        print(x)
    display(out)
    if x.team_match_dict:
        choose_match_dropdown = widgets.interactive(showMatchInfo, x=widgets.Dropdown(description="Match",
                                                                                         options=x.getMatchDictByOpponent()))
        display(choose_match_dropdown)
        widgets.VBox([choose_team_dropdown, out, choose_match_dropdown, match_info_out])
        


choose_team_dropdown = widgets.interactive(showTeamInfo, x =widgets.Dropdown(description="Team", options=team_dict))
display(choose_team_dropdown)

A Jupyter Widget